In [1]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
import os
from concurrent.futures import ThreadPoolExecutor
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import threading

In [2]:
num_objects_json = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/objects')

In [3]:
num_objects = num_objects_json.json()
total_objects = num_objects['total']
object_IDs = num_objects['objectIDs']
object_IDs

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 56,
 57,
 58,
 59,
 62,
 63,
 64,
 65,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 145,
 146,
 147,
 148,
 150,
 151,
 154,
 165,
 166,
 170,
 171,
 172,
 173,
 174,
 176,
 177,
 179,
 180,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 208,
 209,
 211,
 212,
 213,
 214,
 215,
 216,
 218,
 2

In [8]:
url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/'

error_count = 0
last_error_print_time = 0
error_lock = threading.Lock()

def requests_met_api(i):
    global error_count, last_error_print_time
    while True:
        try:
            response = requests.get(url + str(i))
            response.raise_for_status()
            return response.json()
        except Exception:
            with error_lock:
                current_time = time.time()
                if current_time - last_error_print_time >= 60:
                    print('('+str(error_count)+')', 'Error, retrying in 1 minutes')
                    error_count += 1
                    last_error_print_time = current_time
            time.sleep(60)

start_time = time.perf_counter()
count_time = start_time

# add all dictionary responses to a list, will be converted to a DataFrame later
object_list = []

# run multiple requests at once
with ThreadPoolExecutor(max_workers=100) as executor:
    for count, result in enumerate(executor.map(requests_met_api, object_IDs), start=1):

        # print counter every 1000 reponses and how long it took
        if count % 1000 == 0:
            print('count:', str(count), '  time:', str(time.perf_counter() - count_time), '  total time', str(time.perf_counter() - start_time))
            count_time = time.perf_counter()

        object_list.append(result)

# convert the list of dicionatry responses to a DataFrame
df_objects = pd.DataFrame(object_list)
df_objects.to_csv('image_information.csv')

count: 1000   time: 10.311158099997556   total time 10.311161999998149
count: 2000   time: 3.179048900012276   total time 13.491029900003923
count: 3000   time: 5.199705499995616   total time 18.69083320000209
count: 4000   time: 3.3755008000007365   total time 22.066421600000467
count: 5000   time: 4.949459700001171   total time 27.015981899996405
count: 6000   time: 4.128117799991742   total time 31.144186899997294
count: 7000   time: 4.573045800003456   total time 35.71744910000416
count: 8000   time: 3.972867499993299   total time 39.69159849999414
count: 9000   time: 3.994337000011001   total time 43.68641130000469
(0) Error, retrying in 1 minutes
(1) Error, retrying in 1 minutes
(2) Error, retrying in 1 minutes
(3) Error, retrying in 1 minutes
(4) Error, retrying in 1 minutes
count: 10000   time: 245.1770669999969   total time 288.86357279999356
(5) Error, retrying in 1 minutes
count: 11000   time: 126.15059279999696   total time 415.0142055000033
count: 12000   time: 4.003616000

In [5]:
df_objects = pd.read_csv('image_information.csv')
df_objects.head()

C:\Users\Jona\AppData\Local\Temp\ipykernel_10280\1796022690.py:1: DtypeWarning: Columns (4,13,14,15,16,17,19,25,26,37,38,39,40,41,42,43,44,45,46,47,48,49,55,57) have mixed types. Specify dtype option on import or set low_memory=False.
  df_objects = pd.read_csv('image_information.csv')


,Unnamed: 0,objectID,isHighlight,accessionNumber,accessionYear,isPublicDomain,primaryImage,primaryImageSmall,additionalImages,constituents,...,classification,rightsAndReproduction,linkResource,metadataDate,repository,objectURL,tags,objectWikidata_URL,isTimelineWork,GalleryNumber
0,0,1,False,1979.486.1,1979.0,False,NaN,NaN,[],"[{'constituentID': 164292, 'role': 'Maker', 'n...",...,NaN,NaN,NaN,2024-08-14T04:52:51.407Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/search/1,NaN,NaN,False,NaN
1,1,2,False,1980.264.5,1980.0,False,NaN,NaN,[],"[{'constituentID': 1079, 'role': 'Maker', 'nam...",...,NaN,NaN,NaN,2024-08-14T04:52:51.407Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/search/2,NaN,NaN,False,NaN
2,2,3,False,67.265.9,1967.0,False,NaN,NaN,[],NaN,...,NaN,NaN,NaN,2021-04-06T04:41:04.967Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/search/3,NaN,NaN,False,NaN
3,3,4,False,67.265.10,1967.0,False,NaN,NaN,[],NaN,...,NaN,NaN,NaN,2024-01-10T04:57:19.843Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/search/4,NaN,NaN,False,NaN
4,4,5,False,67.265.11,1967.0,False,NaN,NaN,[],NaN,...,NaN,NaN,NaN,2024-01-10T04:57:19.843Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/search/5,NaN,NaN,False,NaN


In [6]:
df_objects.shape[0]

493864